# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
print('modules imported')

modules imported


In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Problem 1

Scrape Website for data and create/clean dataframe

In [3]:
from bs4 import BeautifulSoup
import requests
print('modules imported')

modules imported


In [4]:
#Scraping with BS4

web_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(web_url).text
soup = BeautifulSoup(source,'lxml')
table = soup.find('table')

print('Success website scraped')

Success website scraped


In [5]:
#So the read_html leaves the table as type list
#I feel like keeping it as pandas DataFrame would be easier . . .

readme_html = pd.read_html(str(table))
df = pd.DataFrame(readme_html[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
#drop NA's

df.replace('Not assigned', np.NAN, inplace= True) # The Set Up
df.dropna(inplace= True) # The Committed Crime
df.head() # The Evidence

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [7]:
#starting to clean the data up a little

temp_df = df.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)
df1 = temp_df.reset_index()
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [8]:
df1.shape

(102, 3)

# Problem 2

Adding longitude and latitude. 

The Geocoder Module did not work for me, so I used the the link
provided by the course to load in the data instead


In [9]:
# loading CSV

!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_location = pd.read_csv('Toronto_long_lat_data.csv')
df_location

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
# Renaming Columns to prepare for merge

df_location.columns=['Postal Code', 'Latitude', 'Longitude']
df_location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df1.columns=['Postal Code','Borough','Neighborhood']
df1.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [12]:
final_df = pd.merge(df1, df_location, on='Postal Code')
final_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [13]:
final_df.shape

(102, 5)

# Problem 3

Map and/or explore Neighborhoods

In [14]:
from geopy.geocoders import Nominatim
import folium

print('modules imported')

modules imported


In [15]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
longitude = location.longitude
latitude = location.latitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


In [16]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto